In [4]:
from os import listdir, mkdir, system
from os.path import isfile, isdir, join, exists
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# fix random seed for reproducibility
np.random.seed(7)
fs = 300
input_dir = 'One_Hot_Data/'

## Loading Data

In [11]:
for f in listdir(input_dir):
    print f[0]
    df = pd.read_csv(join(input_dir,f), header=None)
    data = df.values
    X = data[:,:-4]
    X.shape
    X = X.reshape(-1, X.shape[1], 1)
    y = data[:,-3:]
    print (X.shape)
    print (y.shape)
    break

6


KeyboardInterrupt: 

# Split Data

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1, shuffle=True)

In [18]:
from keras import Sequential
from keras.layers import Conv1D, BatchNormalization, Dropout, MaxPooling1D, Flatten, Dense
from keras.callbacks import EarlyStopping

In [11]:
# import keras
# from keras.models import Model
# from keras.layers import Input, Dense, , ,Activation, 
# , ModelCheckpoint
# from keras.utils import plot_model
# from keras import backend as K
# from keras.callbacks import Callback,warnings

In [19]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=(180), activation='relu', input_shape=(X.shape[1],1), 
                 padding='same', strides=1, kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2, strides=2))
model.add(Conv1D(filters=16, kernel_size=(90), activation='relu', 
                 padding='same', strides=1, kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2, strides=2))
# model.add(Conv1D(filters=4, kernel_size=(45), activation='relu', 
#                  padding='same', strides=1, kernel_initializer='he_normal'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(MaxPooling1D(pool_size=2, strides=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 720, 64)           11584     
_________________________________________________________________
batch_normalization_5 (Batch (None, 720, 64)           256       
_________________________________________________________________
dropout_5 (Dropout)          (None, 720, 64)           0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 360, 64)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 360, 16)           92176     
_________________________________________________________________
batch_normalization_6 (Batch (None, 360, 16)           64        
_________________________________________________________________
dropout_6 (Dropout)          (None, 360, 16)           0         
__________

In [21]:
history = model.fit(X_train, y_train, batch_size=100, epochs=2, verbose=1, validation_data=(X_val, y_val), 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, verbose=1)])

Train on 6454 samples, validate on 718 samples
Epoch 1/2
6454/6454 [==============================] - 56s 9ms/step - loss: 0.2789 - acc: 0.9335 - val_loss: 0.2627 - val_acc: 0.9415
Epoch 2/2
6454/6454 [==============================] - 54s 8ms/step - loss: 0.0652 - acc: 0.9678 - val_loss: 0.2334 - val_acc: 0.9429


In [22]:
test_eval = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

797/797 [==============================] - 2s 3ms/step
Test loss: 0.1934191547567901
Test accuracy: 0.9523212038438622


In [33]:
accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.figure(figsize=(20,5))
plt.plot(epochs, accuracy, label='Training accuracy')
plt.plot(epochs, val_accuracy, label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('acc.png',dpi=250)
plt.close()
plt.figure(figsize=(20,5))
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('loss.png',dpi=250)
plt.close()

In [24]:
model.save("model_1.h5py")

In [32]:
new_y_pred = model.predict(X_test)
y_pred = np.zeros((new_y_pred.shape[0],new_y_pred.shape[1]))
y_pred_arg = np.argmax(np.round(new_y_pred),axis=1)
y_test_arg = np.argmax(np.round(y_test),axis=1)
for i in range(y_pred.shape[0]):
    y_pred[i][y_pred_arg[i]] = 1

print (classification_report(y_test, y_pred))
print (confusion_matrix(y_pred_arg, y_test_arg))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       732
           1       0.50      0.29      0.37        34
           2       0.54      0.84      0.66        31

   micro avg       0.95      0.95      0.95       797
   macro avg       0.68      0.71      0.67       797
weighted avg       0.95      0.95      0.95       797
 samples avg       0.95      0.95      0.95       797

[[722   3   4]
 [  9  10   1]
 [  1  21  26]]


In [64]:
new_y_pred = model.predict(X_test)
y_pred = np.zeros((new_y_pred.shape[0],new_y_pred.shape[1]))
y_pred_arg = np.argmax(np.round(new_y_pred),axis=1)
y_test_arg = np.argmax(np.round(y_test),axis=1)
for i in range(y_pred.shape[0]):
    y_pred[i][y_pred_arg[i]] = 1

print (classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90        95
           1       0.88      0.33      0.48        42
           2       0.65      0.74      0.69        38

   micro avg       0.78      0.78      0.78       175
   macro avg       0.78      0.69      0.69       175
weighted avg       0.80      0.78      0.75       175
 samples avg       0.78      0.78      0.78       175



In [ ]:
81% recall. That means the model correctly identified 81% of the total bad loans. That’s pretty great. 
But is this actually representative of how the model will perform? To find out, I’ll calculate the accuracy 
and recall for the model on the test dataset I created initially.
By oversampling before splitting into training and validation datasets, I “bleed” information from the
validation set into the training of the model.

To see how this works, think about the case of simple oversampling (where I just duplicate observations). 
If I upsample a dataset before splitting it into a train and validation set, I could end up with the same 
observation in both datasets. As a result, a complex enough model will be able to perfectly predict the value 
for those observations when predicting on the validation set, inflating the accuracy and recall.

When upsampling using SMOTE, I don’t create duplicate observations. However, because the SMOTE algorithm uses
the nearest neighbors of observations to create synthetic data, it still bleeds information. If the nearest 
neighbors of minority class observations in the training set end up in the validation set, their information 
is partially captured by the synthetic data in the training set. Since I’m splitting the data randomly, we’d
expect to have this happen. As a result, the model will be better able to predict validation set values than
completely new data.
